In [12]:
import requests
import json
from pandas.io.json import json_normalize
import pandas as pd
import requests
from bs4 import BeautifulSoup
import urllib.request

#運用網址取得內容
def getsoup():   
    url = str(row['url'])
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    return soup

#爬取網站內容，並依<p>將所有內容合在一起
def getcontent(x):
    for data in x.find_all('p'):         #將一篇文章中找出有標籤(<p>)的內容
        news_content = data.text  
        conclude.extend([news_content])  #將所有<p>中的資料合併在一起
    content_test = ' '.join(conclude)    #因為會有多個資料，但型態是 '','',''，所以要將這些資料都merge成一個
    l6.extend([content_test])            #最後將資料放入l6的陣列中
    return l6

df = pd.read_excel("fintech資料0509_2000.xlsx", columns = 'post') 

l6 = [] #新聞內文

for i, row in df.iterrows(): 
    conclude = []
    if row['sitename'] == 'UDN': #根據不同網站名稱進行爬蟲
        try:
            x = getsoup().find(id = 'story')
            getcontent(x)
            
        except:
            content = '該頁被刪除'
            l6.extend([content])
          
    elif row['sitename'] == 'Yahoo奇摩股市':
        x = getsoup().find(id = 'aritcletable')
        getcontent(x)
        
    elif row['sitename'] == '富聯網': 
        x = getsoup().find(id = 'NewsMainContent')
        getcontent(x)
        
    elif row['sitename'] == 'Chinatimes':
        x = getsoup().find('div', class_ = 'article-body')      
        getcontent(x)
        
    elif row['sitename'] == '工商時報':
        x = getsoup().find('div', class_ = 'article-body')      
        getcontent(x)
        
    elif row['sitename'] == 'CNA產業新聞':
        x = getsoup().find(id = 'textContent')
        l6.extend([x])
        
    elif row['sitename'] == 'Yam':
        x = getsoup().find(class_ = 'newsArticle')      
        getcontent(x)
        
    elif row['sitename'] == 'Smart智富網':
        x = getsoup().find(class_ = "kidssearchfont13")
    
        for y in x:
            for text in y.find_next_siblings(text=True):
                news_content = text.strip()
                conclude.extend([news_content])
        content_test = ' '.join(conclude)
        l6.extend([content_test]) 
    
    elif row['sitename'] == 'Hinet新聞':
        try:
            x = getsoup().find('div',class_ = 'newsContent')
            l6.extend([x.string])
        except:
            content = '該頁被刪除'
            l6.extend([content])
            
    elif row['sitename'] == 'PChome基金':
        url = str(row['url'])  
        r = requests.get(url)
        r.encoding='UTF-8' #解碼
        soup = BeautifulSoup(r.text, 'html.parser')
        x = soup.find(class_ = "t1320") 
        text = x.find_next_siblings(text=True)
        l6.extend([text])
        
    else:
        content = 'NONE'
        l6.extend([content])

In [13]:
#l1,l2,l3,l4的資料數量都必須一致，才能進行整合!!!

from pandas import DataFrame

df = pd.read_excel("fintech資料0509_2000.xlsx", columns = 'post') 

l1 = []  #公司名稱
l2 = []  #新聞來源
l3 = []  #發布時間
l4 = []  #新聞標題
l5 = []  #新聞url

for i, row in df.iterrows():
    a = row['company']
    l1.extend([a])
    b = row['sitename']
    l2.extend([b])
    c = row['time']
    l3.extend([c])    
    d = row['title']
    l4.extend([d])
    e = row['url']
    l5.extend([e])

df = DataFrame({'company': l1, 'sitename': l2, 'time': l3, 'title': l4, 'url': l5, 'content': l6})
df.to_excel('新聞內文0509.xlsx')